In [1]:
import os
import ee
from geeml.extract import extractor
# import geemap
import os
from google.cloud import storage
from google.cloud import client
import random 

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
# ee.Authenticate()
ee.Initialize(credentials)#h high-volume end-point
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [2]:
dem = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic") #arctic dem
sent_2A = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
ak = ee.FeatureCollection("users/spotter/alaska") #ak shapefile
# lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_lfdb_1985") #ak fire polygons
# lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_lfdb_2014") #ak fire polygons
lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/nbac_1985") #ak fire polygons

# lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/nbac_2013") #ak fire polygons
# lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_mtbs_2014")
# lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/ak_mtbs_1985")

water = ee.ImageCollection("JRC/GSW1_3/YearlyHistory") #water mask
# lfdb = lfdb.filter(ee.Filter.gte('Year', 2014))
print(lfdb.size().getInfo())

11253


In [4]:
terra = ee.ImageCollection("MODIS/061/MOD09A1")
aqua = ee.ImageCollection("MODIS/061/MYD09A1")


modis = terra.merge(aqua)

# Utility function to extract bitmask values
def bitwiseExtract(value, fromBit, toBit=None):
    if toBit is None:
        toBit = fromBit
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)

def mask(image):
    
    # Select StateQA band
    qa = image.select('StateQA')
    # Extract cloud state, cloud shadow state, and cirrus state
    cloudState = bitwiseExtract(qa, 0, 1)
    cloudShadowState = bitwiseExtract(qa, 2)
    cirrusState = bitwiseExtract(qa, 8, 9)

    # Create mask
    mask = cloudState.eq(0).And(cloudShadowState.eq(0)).And(cirrusState.eq(0))

    # Apply mask to the image
    maskedImage = image.updateMask(mask)
    
    return(maskedImage)

    return(maskedImage)


modis = modis.map(mask)


In [ ]:
all_dates = ee.List(lfdb.distinct(["ID"]).aggregate_array("ID"))
all_dates = all_dates.getInfo()
# all_dates = [2173, 2174]
#all_dates = [2173]
# all_dates = [2]
# all_dates = [8017]

for id in all_dates:

  # print(id)
  # if id not in [1909, 1066, 1716]:
  # if id in [3823]:

    try:
        # print(raw_bands.toFloat().getInfo())
        fname = f"modis_{id}.tif"

        # if os.path.isfile(os.path.join('gs://smp-scratch/test_cnn', fname)) == False:

        # name = os.path.join('gs://smp-scratch', fname)
        name = fname

        storage_client = storage.Client()
        # bucket_name = 'smp-scratch/mtbs_1985'
        bucket_name = 'smp-scratch'

        bucket = storage_client.bucket(bucket_name)
        stats = storage.Blob(bucket=bucket, name=name).exists(storage_client)
        if stats == False:
            # print(id)
            #get the sub shape
            sub_shape = lfdb.filter(ee.Filter.eq("ID", id))

            #get all other fire ids that are not this one
            not_fires = lfdb.filter(ee.Filter.neq("ID", id))

            #get the year and date
            this_year = ee.Number(sub_shape.aggregate_array('Year').get(0))
            
            # if this_year >=2001:

            #date ranges for pre and post
            pre_start = ee.Date.fromYMD(this_year.subtract(1), 6, 1)
            pre_end = ee.Date.fromYMD(this_year.subtract(1), 8, 31)
            post_start = pre_start.advance(2, 'year')
            post_end = pre_end.advance(2, 'year')


             #get the centroid of the polygon
            final_buffer = sub_shape.geometry().bounds()

            #get area of bounding box, in square kilometers
            poly_area = final_buffer.area(maxError = 1, proj ='EPSG:3413').divide(1000 * 1000)

            #offset the final_buffer by a random direction of 0.2 degrees
            all_rands = [0.00, 0.02, -0.02]

            rand1 = random.sample(all_rands, 1)[0]
            rand2 = random.sample(all_rands, 1)[0]

            #offset
            proj = ee.Projection("EPSG:4326").translate(rand1, rand2)
            final_buffer = ee.Geometry.Polygon(final_buffer.coordinates(), proj).transform(proj)
            final_buffer2 = final_buffer.buffer(distance= 2000).bounds()#.transform(proj='SR-ORG:6974', maxError=1)


            final_buffer = final_buffer.buffer(distance= 70000).bounds()#.transform(proj='SR-ORG:6974', maxError=1)

            #water mask
            waterOcc = ee.Image("JRC/GSW1_0/GlobalSurfaceWater").clip(final_buffer).select('occurrence')
            jrc_data0 = ee.Image("JRC/GSW1_0/Metadata").select('total_obs').lte(0)
            waterOccFilled = waterOcc.unmask(0).max(jrc_data0)
            waterMask = waterOccFilled.lt(50);

            startYear = pre_start.get('year')

            #convert to client side
            startYear = startYear.getInfo()  # local string
            endYear = str(int(startYear) + 2)
            startYear = str(startYear)

            startDay  = '01-01' # what is the beginning of date filter | month-day
            endDay     = '12-30' # what is the end of date filter | month-day

            pre_input_collection = modis.filterDate(pre_start, pre_end).select(['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07'])

            pre_input = pre_input_collection.median().clip(final_buffer)

            #post firre
            post_input_collection = modis.filterDate(post_start, post_end).select(['sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03', 'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07'])


            post_input = post_input_collection.median().clip(final_buffer)

            # print(post_input.getInfo())
            #get pre and post NDVI, ndvi and ndii
            preNBR = pre_input.normalizedDifference(['sur_refl_b02', 'sur_refl_b07']).select([0], ['preNBR'])


            postNBR = post_input.normalizedDifference(['sur_refl_b02', 'sur_refl_b07']).select([0], ['postNBR'])

            #get dNDVI
            dNBR = preNBR.select('preNBR').subtract(postNBR.select('postNBR'))
            dNBR = dNBR.select('preNBR').rename('dNBR').multiply(1000)


            #-----NDVI
            preNDVI = pre_input.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).select([0], ['preNDVI'])


            postNDVI = post_input.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).select([0], ['postNDVI'])

            #get dNDVI
            NDVI = preNDVI.select('preNDVI').subtract(postNDVI.select('postNDVI'))
            NDVI = NDVI.select('preNDVI').rename('NDVI').multiply(1000)


            #-----NDII
            preNDII = pre_input.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).select([0], ['preNDII'])


            postNDII = post_input.normalizedDifference(['sur_refl_b02', 'sur_refl_b06']).select([0], ['postNDII'])

            #get dNDVI
            NDII = preNDII.select('preNDII').subtract(postNDII.select('postNDII'))
            NDII = NDII.select('preNDII').rename('NDII').multiply(1000)

            #we need to see which image ids from the entire lfdb are already included in the buffer
            lfdb_filtered_orig = lfdb.filterBounds(final_buffer)

            #ensure all fires are within the actual year of interest (this_year) and two years prior, otherwise ignore

            first_year =  int(startYear) + 1
            second_year =  int(startYear)
            third_year =  int(startYear) - 1
            fourth_year = int(startYear) + 2

            lfdb_filtered = lfdb_filtered_orig.filter(ee.Filter.eq("Year", first_year))

            bad_filtered = lfdb_filtered_orig.filter(ee.Filter.Or(ee.Filter.eq("Year", second_year), ee.Filter.eq("Year", third_year), ee.Filter.eq("Year", fourth_year)))


            #get ids which are in image
            all_dates_new = ee.List(lfdb_filtered.distinct(["ID"]).aggregate_array("ID")).getInfo()


            #remove ids from all dates which we do not need anymore
            all_dates = [i for i in all_dates if i not in all_dates_new]


            #now where the fire is turn the values to 0 and where the fire isn't turn them to 0
            # fire_rast = sub_shape.filter(ee.Filter.notNull(['ID'])).reduceToImage(properties = ['ID'], reducer =  ee.Reducer.first())

            fire_rast = lfdb_filtered.reduceToImage(properties= ['ID'], reducer = ee.Reducer.first())

            #bad fire rast
            bad_fire_rast = bad_filtered.reduceToImage(properties= ['ID'], reducer = ee.Reducer.first())

            #change values
            fire_rast = fire_rast.where(fire_rast.gt(0), 1)

            #chabnge values for bad fire raster 
            bad_fire_rast = bad_fire_rast.where(bad_fire_rast.gt(0), 1)

            #if the fires overlap we want to keep those locations
            bad_fire_rast = bad_fire_rast.where(bad_fire_rast.eq(1).And(fire_rast.eq(1)), 2).unmask(-999)

            # #fire_locaation values to 1 by first copying ndvi
            fire_rast = fire_rast.rename(['y'])

            #copy the first values of raw_bands
            y = post_input.select(['sur_refl_b02'], ['y'])#.toShort()

            #turn all values of y to 0
            y  = y.where(y.gt(0), 0)

            #turn values to 1 where fire_rast is 1
            y  = y.where(fire_rast.eq(1), 1)

            #y  = y.where(fire_rast.eq(1).And(dNBR.gte(0)), 1)

            # y  = y.where(dNBR.gte(84), 1)


            #difference the raw bands
            diff = pre_input.subtract(post_input)

            #combine

            raw_bands = diff.addBands(dNBR).addBands(NDVI).addBands(NDII)

            b1 = raw_bands.select('sur_refl_b01').cast({'sur_refl_b01':'short'})
            b2 = raw_bands.select('sur_refl_b02').cast({'sur_refl_b02':'short'})
            b3 = raw_bands.select('sur_refl_b03').cast({'sur_refl_b03':'short'})
            b4 = raw_bands.select('sur_refl_b04').cast({'sur_refl_b04':'short'})
            b5 = raw_bands.select('sur_refl_b05').cast({'sur_refl_b05':'short'})
            b6 = raw_bands.select('sur_refl_b06').cast({'sur_refl_b06':'short'})
            b7 = raw_bands.select('sur_refl_b07').cast({'sur_refl_b07':'short'})

            b8 = raw_bands.select('dNBR').cast({'dNBR':'short'})
            b9 = raw_bands.select('NDVI').cast({'NDVI':'short'})
            b10 = raw_bands.select('NDII').cast({'NDII':'short'})
            # b10 = raw_bands.select('y').cast({'y':'float'})
            b11 = y.select('y').cast({'y':'short'})


            # b1 = raw_bands.select('sur_refl_b01').cast({'sur_refl_b01':'float'})
            # b2 = raw_bands.select('sur_refl_b02').cast({'sur_refl_b02':'float'})
            # b3 = raw_bands.select('sur_refl_b03').cast({'sur_refl_b03':'float'})
            # b4 = raw_bands.select('sur_refl_b04').cast({'sur_refl_b04':'float'})
            # b5 = raw_bands.select('sur_refl_b05').cast({'sur_refl_b05':'float'})
            # b6 = raw_bands.select('sur_refl_b06').cast({'sur_refl_b06':'float'})
            # b7 = raw_bands.select('sur_refl_b07').cast({'sur_refl_b07':'float'})

            # b8 = raw_bands.select('dNBR').cast({'dNBR':'float'})
            # b9 = raw_bands.select('NDVI').cast({'NDVI':'float'})
            # b10 = raw_bands.select('NDII').cast({'NDII':'float'})
            # #b10 = raw_bands.select('y').cast({'y':'float'})
            # b11 = y.select('y').cast({'y':'float'})




            raw_bands = b1.addBands(b2).addBands(b3).addBands(b4).addBands(b5).addBands(b6).addBands(b7).addBands(b8).addBands(b9).addBands(b10)
            # raw_bands = raw_bands.updateMask(waterMask)
            raw_bands = raw_bands.updateMask(waterOccFilled.lt(50).And(bad_fire_rast.neq(1)))
            raw_bands = raw_bands.addBands(b11)


            task = ee.batch.Export.image.toCloudStorage(
                              image = raw_bands.toShort(),
                              region=final_buffer2, 
                              # region=raw_bands.geometry(), 

                              description='modis_' + str(id),
                              # crs= 'SR-ORG:6974'
                              crs = 'EPSG:3413',
                              scale = 463.3127165275,
                              maxPixels=1e13,
                              bucket = 'smp-scratch')

            task.start()
            print(id)
            
    except:
        pass


 

2497
4628
6169
10045
10849
11241
11146
471
636
697
728
1067
2227
2576
2514
2534
3606
3635
3645
3685
3703
4541
4721
4490
4568
4573
4480
4589
4633
5319
5422
5322
5419
5455
5330
5432
5523
5525
5531
5520
6559
6448
6357
6383
6322
6324
6329
6336
6337
6373
6400
6432
6565
6610
6611
6334
6375
6439
6330
6397
7174
7084
7406
7150
7156
7230
7272
7410
7438
8443
8442
9082
9086
8486
9072
9455
9463
9807
9633
10043
10630
10621
10481
10646
11183
11201
11207
11237
379
375
50
142
263
389
374
387
95
486
444
578
736
757
787
795
932
1435
1264
1298
1314
1220
1202
1364
1493
1296
1327
1695
1565
1757
1544
1545
1556
1559
1584
1595
1602
1615
1617
1620
1676
1681
1689
1696
1785
1796
1808
1580
1601
1781
1568
1577
1581
1570
1585
1603
1673
1794
1795
1822
1686
1683
1684
1887
1890
1912
1925
1940
1892
1895
1897
1896
1900
1937
1958
1907
1909
1917
1926
1929
1931
1975
1982
1889
1910
1981
2031
2184
2187
2339
2233
2354
2013
2182
2032
2598
2471
2481
2564
2640
2749
3086
3412
3150
3180
3462
3631
3670
3546
3634
3843
3885
3495
3508


In [ ]:
't'

In [28]:
pre_fire.size().getInfo()

NameError: name 'pre_fire' is not defined

In [29]:
this_year.getInfo()

1999

In [5]:
raw_bands.geometry().getInfo()

{'geodesic': False,
 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}},
 'type': 'Polygon',
 'coordinates': [[[-158, 59], [-157, 59], [-157, 60], [-158, 60], [-158, 59]]]}

In [8]:
task = ee.batch.Export.image.toCloudStorage(
                              image = raw_bands.toShort(),
                              # region=final_buffer, 
                              region=raw_bands.geometry(), 

                              description='delete_' + str(id),
                              crs= 'SR-ORG:6974',
                              scale = 463.3127165275,
                              maxPixels=1e13,
                              bucket = 'smp-scratch')

task.start()